In [49]:
import pandas as pd
import sqlite3

# Connecting to the database with a timeout to handle locks
superstore_connection = sqlite3.connect("superstore.db", timeout=10)
crsr = superstore_connection.cursor()

# Set busy timeout and WAL mode
crsr.execute("PRAGMA busy_timeout = 3000;")
print("Connected to the database")

# Load the CSV
superstore_csv = pd.read_csv(
    r"/Users/jaamann/Documents/Vs Code Python /Code Academy/Projects/E-Commerce Project/Sample-Superstore.csv",
    sep=",",
    encoding='latin1',
    on_bad_lines='skip'
)

# Create Customer table
crsr.execute('''
CREATE TABLE IF NOT EXISTS Customer (
    customer_ID TEXT PRIMARY KEY,
    customer_name TEXT,
    segment TEXT,
    country TEXT,
    city TEXT,
    state TEXT,
    postal_code TEXT,
    region TEXT
);
''')

# Create Orders table
crsr.execute('''
CREATE TABLE IF NOT EXISTS Orders (
    customer_ID TEXT,
    order_ID TEXT PRIMARY KEY,
    order_date INTEGER,
    ship_mode TEXT,
    ship_date TIMESTAMP,
    FOREIGN KEY(customer_ID) REFERENCES Customer(customer_ID)
);
''')

# Create Sales table
crsr.execute('''
CREATE TABLE IF NOT EXISTS Sales (
    order_id TEXT PRIMARY KEY,
    product_id TEXT,
    discount REAL,
    profits REAL,
    quantity INTEGER,
    sales REAL,        
    FOREIGN KEY(order_id) REFERENCES Orders(order_ID),
    FOREIGN KEY(product_id) REFERENCES Product(product_id)
);
''')

# Create Product table
crsr.execute('''
CREATE TABLE IF NOT EXISTS Product (
    product_id TEXT PRIMARY KEY,
    category TEXT,
    sub_category TEXT,
    product_name TEXT
);
''')

# Insert data into Customer table
customer_data = superstore_csv[['Customer ID', 'Customer Name', 'Segment', 'Country', 'City', 'State', 'Postal Code', 'Region']]
customer_data.columns = ['customer_ID', 'customer_name', 'segment', 'country', 'city', 'state', 'postal_code', 'region']  # Rename columns
crsr.executemany('''
    INSERT OR IGNORE INTO Customer (customer_ID, customer_name, segment, country, city, state, postal_code, region)
    VALUES (?, ?, ?, ?, ?, ?, ?, ?)
''', customer_data.values.tolist())

# Insert data into Orders table
orders_data = superstore_csv[['Customer ID', 'Order ID', 'Order Date', 'Ship Mode', 'Ship Date']]
orders_data.columns = ['customer_ID', 'order_ID', 'order_date', 'ship_mode', 'ship_date']  # Rename columns
crsr.executemany('''
    INSERT OR IGNORE INTO Orders (customer_ID, order_ID, order_date, ship_mode, ship_date)
    VALUES (?, ?, ?, ?, ?)
''', orders_data.values.tolist())

# Insert data into Sales table
sales_data = superstore_csv[['Order ID', 'Product ID', 'Discount', 'Profit', 'Quantity', 'Sales']]
sales_data.columns = ['order_id', 'product_id', 'discount', 'profits', 'quantity', 'sales']  # Rename columns
crsr.executemany('''
    INSERT OR IGNORE INTO Sales (order_id, product_id, discount, profits, quantity, sales)
    VALUES (?, ?, ?, ?, ?, ?)
''', sales_data.values.tolist())

# Insert data into Product table
product_data = superstore_csv[['Product ID', 'Category', 'Sub-Category', 'Product Name']]
product_data.columns = ['product_id', 'category', 'sub_category', 'product_name']  # Rename columns
crsr.executemany('''
    INSERT OR IGNORE INTO Product (product_id, category, sub_category, product_name)
    VALUES (?, ?, ?, ?)
''', product_data.values.tolist())

# Commit the changes and close the connection
superstore_connection.commit()
#superstore_connection.close()

print("Data inserted successfully into the database.")


Connected to the database
Data inserted successfully into the database.


In [56]:
query = '''
SELECT 
    p.category,
    SUM(s.quantity * s.profits) AS total_revenue
FROM 
    Sales s
JOIN 
    Product p ON s.product_id = p.product_id
GROUP BY 
    p.category
ORDER BY 
    total_revenue DESC
LIMIT 1;
'''

result = crsr.execute(query).fetchone()
print("Category generating the maximum sales revenue:", result[0], "| Total Revenue:", result[1])


Category generating the maximum sales revenue: Technology | Total Revenue: 342648.2553


In [51]:
query = '''
SELECT 
    p.category,
    SUM(s.profits) AS total_profit
FROM 
    Sales s
JOIN 
    Product p ON s.product_id = p.product_id
GROUP BY 
    p.category
HAVING 
    total_profit < 0;
'''

results = crsr.execute(query).fetchall()
if results:
    print("Categories making a loss:")
    for row in results:
        print("Category:", row[0], " | Total Profit:", row[1])
else:
    print("No categories are making a loss.")


No categories are making a loss.


In [52]:
# Query for top 5 states with maximum revenue
query_max = '''
SELECT 
    c.state,
    SUM(s.quantity * s.profits) AS total_revenue
FROM 
    Sales s
JOIN 
    Orders o ON s.order_id = o.order_ID
JOIN 
    Customer c ON o.customer_ID = c.customer_ID
GROUP BY 
    c.state
ORDER BY 
    total_revenue DESC
LIMIT 5;
'''

max_results = crsr.execute(query_max).fetchall()
print("Top 5 States Generating Maximum Revenue:")
for row in max_results:
    print("State:", row[0], "| Total Revenue:", row[1])

# Query for bottom 5 states with minimum revenue
query_min = '''
SELECT 
    c.state,
    SUM(s.quantity * s.profits) AS total_revenue
FROM 
    Sales s
JOIN 
    Orders o ON s.order_id = o.order_ID
JOIN 
    Customer c ON o.customer_ID = c.customer_ID
GROUP BY 
    c.state
ORDER BY 
    total_revenue ASC
LIMIT 5;
'''

min_results = crsr.execute(query_min).fetchall()
print("\nBottom 5 States Generating Minimum Revenue:")
for row in min_results:
    print("State:", row[0], "| Total Revenue:", row[1])


Top 5 States Generating Maximum Revenue:
State: California | Total Revenue: 140723.0224
State: New York | Total Revenue: 131445.8991
State: Texas | Total Revenue: 70208.0016
State: Arizona | Total Revenue: 67542.3648
State: Washington | Total Revenue: 37729.543900000004

Bottom 5 States Generating Minimum Revenue:
State: Florida | Total Revenue: -17649.1734
State: Connecticut | Total Revenue: -9796.7257
State: Michigan | Total Revenue: -2518.8648000000007
State: Kansas | Total Revenue: 89.8341
State: Nevada | Total Revenue: 367.6098


In [53]:
superstore_csv.head(10)

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,...,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit
0,1,CA-2016-152156,11/8/2016,11/11/2016,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.9600,2,0.00,41.9136
1,2,CA-2016-152156,11/8/2016,11/11/2016,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400,3,0.00,219.5820
2,3,CA-2016-138688,6/12/2016,6/16/2016,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,...,90036,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.6200,2,0.00,6.8714
3,4,US-2015-108966,10/11/2015,10/18/2015,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,33311,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775,5,0.45,-383.0310
4,5,US-2015-108966,10/11/2015,10/18/2015,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,33311,South,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.3680,2,0.20,2.5164
5,6,CA-2014-115812,6/9/2014,6/14/2014,Standard Class,BH-11710,Brosina Hoffman,Consumer,United States,Los Angeles,...,90032,West,FUR-FU-10001487,Furniture,Furnishings,Eldon Expressions Wood and Plastic Desk Access...,48.8600,7,0.00,14.1694
6,7,CA-2014-115812,6/9/2014,6/14/2014,Standard Class,BH-11710,Brosina Hoffman,Consumer,United States,Los Angeles,...,90032,West,OFF-AR-10002833,Office Supplies,Art,Newell 322,7.2800,4,0.00,1.9656
7,8,CA-2014-115812,6/9/2014,6/14/2014,Standard Class,BH-11710,Brosina Hoffman,Consumer,United States,Los Angeles,...,90032,West,TEC-PH-10002275,Technology,Phones,Mitel 5320 IP Phone VoIP phone,907.1520,6,0.20,90.7152
8,9,CA-2014-115812,6/9/2014,6/14/2014,Standard Class,BH-11710,Brosina Hoffman,Consumer,United States,Los Angeles,...,90032,West,OFF-BI-10003910,Office Supplies,Binders,DXL Angle-View Binders with Locking Rings by S...,18.5040,3,0.20,5.7825
9,10,CA-2014-115812,6/9/2014,6/14/2014,Standard Class,BH-11710,Brosina Hoffman,Consumer,United States,Los Angeles,...,90032,West,OFF-AP-10002892,Office Supplies,Appliances,Belkin F5C206VTEL 6 Outlet Surge,114.9000,5,0.00,34.4700
